In [4]:

import os
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append('../..')
confluence_username = os.getenv("CONFLUENCE_USERNAME")
confluence_password = os.getenv("CONFLUENCE_PASSWORD")

from llama_index.readers.confluence import ConfluenceReader



In [9]:



base_url = "https://wiki.softtech.com.tr/"
space_keys = ["SPISS"]#, "YNYGAB", "SPISS"],["YPZ"]
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract' 
reader = ConfluenceReader(base_url=base_url,  user_name=confluence_username, password=confluence_password)
spaces = {}
for space in space_keys:
    spaces[space] = reader.load_data(include_attachments=False, space_key = space)



In [11]:

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_milvus import Milvus
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2")
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

abc = 0
for space_key,documents in spaces.items():
    chunks = []
    print(f'page_size : {len(documents)}')
    for document in documents:
        title = document.extra_info['title']
        content = document.text
        source = 'confluence'
        doc_id = document.doc_id
        url = document.extra_info['url']
        # [docs.extend(Document(page_content=chunks[i], metadata = {'source':file.name, 'page': i}) for i in range(len(chunks)))]
        metadata = {'title': title, 
                'source': source, 
                'doc_id': doc_id, 
                'url': url,
                'Header_1': '', 
                'Header_2': ''
                }
        
        md_header_splits = markdown_splitter.split_text(content)
        if abc == 0:
            for md in md_header_splits:
                print(md)
            abc = 1
        for i, split in enumerate(md_header_splits):
            # print("sub_id den önce", metadata)
            metadata['sub_id'] = i
            metadata.update(split.metadata)
            # print("sub_id den sonra", metadata)
            if not (metadata['Header_1'] or metadata['Header_2']):
                page_content = f"Introduction to {metadata['title']}:\n" + split.page_content
            else:
                page_content = f"{metadata['title']}\n\tsubsection:{metadata['Header_1']}:\n\tsub_subsection:{metadata['Header_2']}:\n" + split.page_content
            chunks.append(Document(page_content=page_content, metadata = metadata))
        # if i == 1:
        #     pprint(chunks)
        #     i == 2
    print(f'chunk size: {len(chunks)}')
    vectorstore = Milvus.from_documents(
        documents=chunks,
        embedding=embeddings,
        # collection_name = f'{space_key}_with_attachments',
        collection_name = space_key,
        connection_args={
            "uri": "./milvus_demo.db",},
        # Override LangChain default values for Milvus.
        consistency_level="Eventually",
        drop_old=True,
        index_params = {
            "metric_type": "COSINE",
            "index_type": "AUTOINDEX",
            "params": {}}
    )
    

    

/home/mehmet/miniconda3/envs/milvus/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


page_size : 448
page_content='Finacials  
Up to 31st of October  
| Customer| Project|  Devlivery|  Planned Effort|  Actual Effort| Remaining
Effort |  Remaining Work| Finish Date | Total Cost | Income | Profit
---|---|---|---|---|---|---|---|---|---|---|---
Future Development|
|  **PS8004/ Finansal Ürünler Back Office Uygulamaları**|
| 6.967| 4.098| 2.869|
|
|
|
|  
|  **Maxis Investment**|
|
|  860|
|
|
|
|
|
|  
|
|
|  **SPMP-62 / Maxis-1**| 540|
|
|
|
|
|
|  
|
|
|  **SPMP-64 / MAXIS-2**| 320|
|
|
|
|
|
|  
|  **İş Portföy**|
|
|  250|
|
|
|
|
|
|  
|
|
|  **SPMP-61 / İPY-1**|  250|
|
|
|
|
|
|  
|  **TKYB**|
|
|  5.237|
|
|
|
|
|
|  
|
|
|  **SPMP-63 / TKYB-1**| 117 |
|
|
|
|
|
|  
|
|
| **SPMP-65 / TKYB-2(hazine)**| 2.620|
|
|
|
|
|
|  
|
|
|  **SPMP-66 /  TKYB-3**| 1.500|
|
|
|
|
|
|  
|  **Softtech**|
|
|  620|
|
|
|
|
|
|  
|
|
|  **SPMP-261 /RallyFrameWork-1**| 300|
|
|
|
|
|
|  
|
|
|  **SPMP-280 /Portera Yönetim Faaliyetleri**|  200|
|
|
|
|
|
|  
|
|
|  **SPMP-415 /portera

In [16]:
space_items = {
    "Ailab": "YPZ",
    "Mimari Yetkinlik Merkezi":"MYM",
    "Sermaye Piyasaları İşlem Sonrası Sistemleri Direktörlüğü": "SPISS" ,
}
# Koleksiyonları yükle
space_names = list(space_items)
space_names[0]

'Ailab'

In [ ]:
# This code sample uses the 'requests' library:
# http://docs.python-requests.org
import requests
from requests.auth import HTTPBasicAuth
import json

url = "https://wiki.softtech.com.tr/wiki/api/v2/spaces"

auth = HTTPBasicAuth("email@example.com", "<api_token>")

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   auth=auth,
   keys = ["YPZ"]
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

In [5]:
import os

# Access a specific environment variable
variable_name = "CONFLUENCE_PASSWORD"
home_directory = os.getenv(variable_name)
print(f"Home directory: {home_directory}")

# List all environment variables
all_variables = os.environ
for key, value in all_variables.items():
    print(f"{key}: {value}")


Home directory: None
CHROME_DESKTOP: code-url-handler.desktop
COLORTERM: truecolor
CONDA_DEFAULT_ENV: milvus
CONDA_EXE: /home/mehmet/miniconda3/bin/conda
CONDA_PREFIX: /home/mehmet/miniconda3/envs/milvus
CONDA_PROMPT_MODIFIER: (milvus) 
CONDA_PYTHON_EXE: /home/mehmet/miniconda3/bin/python
CONDA_SHLVL: 2
DBUS_SESSION_BUS_ADDRESS: unix:path=/run/user/1000/bus,guid=e0c51f1512bcc181be4f7ea166bdd5dc
DBUS_STARTER_ADDRESS: unix:path=/run/user/1000/bus,guid=e0c51f1512bcc181be4f7ea166bdd5dc
DBUS_STARTER_BUS_TYPE: session
DESKTOP_SESSION: ubuntu
DISPLAY: :0
ELECTRON_NO_ATTACH_CONSOLE: 1
GDK_BACKEND: x11
GDK_PIXBUF_MODULEDIR: /snap/code/166/usr/lib/x86_64-linux-gnu/gdk-pixbuf-2.0/2.10.0/loaders
GDK_PIXBUF_MODULE_FILE: /home/mehmet/snap/code/common/.cache/gdk-pixbuf-loaders.cache
GDMSESSION: ubuntu
GIO_MODULE_DIR: /home/mehmet/snap/code/common/.cache/gio-modules
GNOME_DESKTOP_SESSION_ID: this-is-deprecated
GNOME_SETUP_DISPLAY: :1
GNOME_SHELL_SESSION_MODE: ubuntu
GNOME_TERMINAL_SCREEN: /org/gnome/T

In [1]:

import os
print(os.environ)
from dotenv import load_dotenv, find_dotenv
load_dotenv()
import sys
sys.path.append('../..')
confluence_username = os.getenv("CONFLUENCE_USERNAME")
confluence_password = os.getenv("CONFLUENCE_PASSWORD")
print(os.environ.get('CONFLUENCE_PASSWORD'))
from llama_index.readers.confluence import ConfluenceReader


environ({'CHROME_DESKTOP': 'code-url-handler.desktop', 'COLORTERM': 'truecolor', 'CONDA_DEFAULT_ENV': 'milvus', 'CONDA_EXE': '/home/mehmet/miniconda3/bin/conda', 'CONDA_PREFIX': '/home/mehmet/miniconda3/envs/milvus', 'CONDA_PROMPT_MODIFIER': '(milvus) ', 'CONDA_PYTHON_EXE': '/home/mehmet/miniconda3/bin/python', 'CONDA_SHLVL': '2', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus,guid=e0c51f1512bcc181be4f7ea166bdd5dc', 'DBUS_STARTER_ADDRESS': 'unix:path=/run/user/1000/bus,guid=e0c51f1512bcc181be4f7ea166bdd5dc', 'DBUS_STARTER_BUS_TYPE': 'session', 'DESKTOP_SESSION': 'ubuntu', 'DISPLAY': ':0', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'GDK_BACKEND': 'x11', 'GDK_PIXBUF_MODULEDIR': '/snap/code/166/usr/lib/x86_64-linux-gnu/gdk-pixbuf-2.0/2.10.0/loaders', 'GDK_PIXBUF_MODULE_FILE': '/home/mehmet/snap/code/common/.cache/gdk-pixbuf-loaders.cache', 'GDMSESSION': 'ubuntu', 'GIO_MODULE_DIR': '/home/mehmet/snap/code/common/.cache/gio-modules', 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated', 'GNO

In [7]:
# This code sample uses the 'requests' library:
# http://docs.python-requests.org
import requests
from requests.auth import HTTPBasicAuth
import json
id = "YPZ"
url = "https://wiki.softtech.com.tr/wiki/api/v2/spaces/{id}/pages"

auth = HTTPBasicAuth(username = "MEHMET.AYDIN2", password = "Bayar1649.*")

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   auth=auth
)
print(response)
# print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

<Response [500]>


SyntaxError: invalid syntax (3957199109.py, line 1)